# Black-body radiation

An object that is in thermal equilibrium, will emit electromagnetic radiation whose properties are described as "black-body" radiation.

- An object in thermal equilibrium has a single temperature, and does not change with time
- If an object radiates electromagnetic radiation, it loses energy, and its temperature shoud decrease, is that a contradiction?
    - No, it is an idealized situation. As long as the energy emitted is very small, negligible, compared to the total thermal energy contained in a body we can approximate it by a black-body
    - Interestingly this is the situation for stars, like the sun. Even though they emit a large amount of energy (in the case of the sun, enough to warm a planet 150 million kilometers away, sustaining its biosphere and economy), that amount of energy is small to its inner energy (which is fueled by nuclear reactions), its temperature is very stable, and the black-body is a good approximation

The mechanism by which this radiation is generated is the acceleration of its constituent electric charges (electrons) when they change course by the influence of all other charges, as they move with a velocity that depends on the body's temperature.
 

The amount of energy radiated at each frequency / wavelenght is given by the Planck law. It was measured empirically at the end of the XIX century, and Planck started the quantum revolution when he found a way to derive it from first principles.

The law, which you do not need to memorize, can be written as a function of frequency:

$$B_{\nu}(\nu, T) = \frac{2h\nu^3}{c^2}\frac{1}{\exp{\frac{h\nu}{k_B T}}-1}$$

where $B_{\nu}$ is the spectral radiance (in units of power per $\mathrm{Hz}$ per $\mathrm{m}^2$), $h$ is Planck's constant ($h=6.63\times10^-34\,\mathrm{m^2kg/s}$), $c$, the speed of light ($c=3\times10^8\,\mathrm{m/s}$), and $k_B$ Boltzmann's constant ($k_B=1.38\times10^{-23}\mathrm{m^2kg/s^2/K}$). 

Note that for a given frequency $\nu$, the spectral radiance only depends on the temperature of the black-body.

It can also be written as a funcion of wavelength:

$$B_{\lambda}(\lambda, T) = \frac{2h c^2}{\lambda^5}\frac{1}{\exp{\frac{h c}{\lambda k_B T}}-1}$$

If we sum (integrate) the spectral radiance across the spectrum, we obtain the total radiance, which corresponds to the total power emitted by the black-body. Turns out its value is a simple function of the black-body's temperature, named the Stefan-Boltzmann's law (this one, you may want to remember):

$$L = \sigma T^4$$

where $\sigma$ is the Stefan-Boltzmann's constant ($\sigma=5.67\times10^{-8}\,\mathrm{W/m^2/K^4}$), and the total radiance corresponds to the area below the curve of the spectral radiance (see below)

Finally, the frequency (wavelength) at which the spectral radiance peaks, shifts also with the temperature in a simple fashion, called Wien's law (again, useful to remember):

$$\lambda_{\mathrm{max}} = \frac{b}{T}$$

where $b$ is Wien's displacement constant ($b=2898\,\rm{\mu m K}$)

In [1]:
# Basic code to do computations with a black-body

import astropy.units as u
from astropy.constants import c, h, k_B, b_wien, sigma_sb
import numpy as np

class BlackBody:
    def __init__(self,T):
        # A black body is fully defined by its temperature
        try:
            self.T = T.to(u.K)
        except AttributeError:
            self.T = T * u.K
        # We can use Wien's law to find the wavelength of the maximum
        self.lambda_max = (b_wien / self.T).to(u.nm)
        self.freq_max   = (c / self.lambda_max).to(u.Hz)
        # We can use Stefan Boltmann's law to get the total radiance
        self.total_radiance = sigma_sb * self.T**4
            
    def spec_radiance_freq(self, nu, units='cgs'):
        try:
            nu = nu.to(u.Hz)
        except AttributeError:
            nu *= u.Hz
        
        exponent  = (h * nu / k_B / self.T).decompose()
        prefactor = (2 * h * nu**3 / c**2).decompose()
        radiance  = prefactor / (np.exp(exponent) - 1)
        
        if   units=='cgs':
            radiance = radiance.to(u.erg / u.s / u.cm**2 / u.Hz)
        elif units=='SI':
            radiance = radiance.to(u.W / u.m**2 / u.Hz)
        
        return radiance
    
    def spec_radiance_wavelenght(self, lamb, units='cgs'):
        try:
            lamb = lamb.to(u.nm)
        except AttributeError:
            lamb *= u.nm
            
        exponent  = (h * c / lamb / k_B / self.T).decompose()
        prefactor = (2 * h * c**2 / lamb**5).decompose()
        radiance  = prefactor / (np.exp(exponent) - 1)
        
        if units=='cgs':
            radiance = radiance.to(u.erg / u.s / u.m**2 / u.nm)
        elif units=='SI':
            radiance = radiance.to(u.W / u.m**2 / u.nm)
            
        return radiance

In [8]:
# Widget to show effect of change in temperature on the spectral radiance

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

def bb_demo_plot(T, lambda_min, lambda_range):
    # initialize two black bodies, one with the sun's temperature, and the other one
    bb     = BlackBody(T)
    T_ref  = 5778 * u.K
    bb_ref = BlackBody(T_ref)
    
    # compute the radiance
    lambdas = (np.linspace(1e-1, 3, 200) * u.micron).to(u.nm)
    rad     = bb.spec_radiance_wavelenght(lambdas)
    rad_ref = bb_ref.spec_radiance_wavelenght(lambdas)
    
    # pick a range of wavelenghts
    lambda_max  = lambda_min + lambda_range
    old_lambdas = lambdas.value
    new_lambdas = lambdas[np.logical_and(old_lambdas >= lambda_min, old_lambdas <= lambda_max)]
    new_rad     = rad[np.logical_and(old_lambdas >= lambda_min, old_lambdas <= lambda_max)]
    new_rad_ref = rad_ref[np.logical_and(old_lambdas >= lambda_min, old_lambdas <= lambda_max)]
    
    # make the plot
    matplotlib.rcParams.update({'font.size': 16})
    fig = plt.figure(figsize=(15,10))
    ax  = fig.add_subplot(111)
    ax.plot(lambdas, rad_ref, linewidth=3, color='k', alpha=0.50, label=r"{}".format(T_ref))
    ax.plot(lambdas, rad,     linewidth=3, color='b', alpha=1.00, label=r"{}".format(bb.T))
    
    ax.fill_between(new_lambdas, new_rad, 0, color='b', alpha=0.5)
    ax.fill_between(new_lambdas, new_rad_ref, 0, color='k', alpha=0.5)
    
    ax.axvline(bb.lambda_max.value, linewidth=3, linestyle='--', color='b')
    ax.axvline(bb_ref.lambda_max.value, linewidth=3, linestyle='--', color='k', alpha=0.25)
    
    ax.legend(loc=1)
    ax.set_xlabel(r"Wavelength [nm]")
    ax.set_ylabel(r"Spectral radiance [erg / s / m$^2$ / nm]")
    ax.grid()
    ax.set_ylim([0, 1.5e12])
#    ax.set_yscale('log')
    
    plt.tight_layout()
    plt.show()

In [9]:
# Code to do a demo in class on the effec of the temperature of a black body on
# its spectral radiance

from ipywidgets import interactive

bb_demo = interactive(bb_demo_plot, T=(2e3, 8e3, 100), lambda_min=(0.1, 2000, 100), lambda_range=(0, 1000, 50))
output  = bb_demo.children[-1]
bb_demo

interactive(children=(FloatSlider(value=5000.0, description='T', max=8000.0, min=2000.0, step=100.0), FloatSli…

Note a series of observations:

- Spectral radiances do not cross. The spectral radiance of a hotter black-body will always be above that of a colder black-body.
- The peak of the radiance moves towards smaller wavelengths (higher frequencies, higher photon energies, bluer light) as the temperature of the black-body increases.
- The total radiance (area below spectral radiance) increases very fast as the temperature of the black-body increases.